In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
batting_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Batting.csv'
salary_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Salaries.csv'
people_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/People.csv'
fielding_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Fielding.csv'
pitching_url = 'https://raw.githubusercontent.com/SeanG347/LahmanBaseballDatabase/main/Pitching.csv'

batting_df = pd.read_csv(batting_url)
salary_df = pd.read_csv(salary_url)
people_df = pd.read_csv(people_url)
fielding_df = pd.read_csv(fielding_url)
pitching_df = pd.read_csv(pitching_url)

In [3]:
salary_df['season'] = salary_df.groupby('playerID').cumcount() + 1

In [4]:
salary_df.head()

,yearID,teamID,lgID,playerID,salary,season
0,2004,SFN,NL,aardsda01,300000,1
1,2007,CHA,AL,aardsda01,387500,2
2,2008,BOS,AL,aardsda01,403250,3
3,2009,SEA,AL,aardsda01,419000,4
4,2010,SEA,AL,aardsda01,2750000,5


In [5]:
# Contract model for batting
# Note for the model we must use a normalized salary as the average salaries have varied significantly year over year.
important_features = ['AVG','OPS','HR','bats','POS','Age','ZR/dWAR']
# So we have to find a metric to determine defensive effectiveness.
# Since this model will focus on free-agency contracts, not arbitration, we have to 
# find a way to determine when a player has signed a free agent contract.

Ideas for free agent contract determination:
* The salaries players make before signing in free agency are almost always relatively low, i.e., below the mean. This is especially true in pre-arbitration years, however, some players, i.e., Bo Bichette in 2025, made well over the league average salary, this will have to be accounted for.
* Percentage increase from previous salary, most free agents will make some amount more than they did the previous year (last year of arbitration).
* Could potentially simply work out when the player would be free-agency eligible based on the arbitration rules, and then only include data from the arbitration/free-agency era (which we should do anyways as it is more relevant to the current free-agency market).

Notes:
   * This model needs to take in the player data from the years leading up to a free agent contract, and then extrapolate what it learned from each player to a new player. The goal for this is to have this done before Kyle Tucker and Bo Bichette sign contracts this offseason, and see how it does on those predictions. 
   * Incorporating contract term is another factor.